# Project description
Type 1 diabetes is a disease which results from loss of the beta cells within the pancreas.  Such cells produce insulin, which controls the glucose level in your blood.  When glocuse is too high one feels sick, and verve damage is possible.  When glucose is too low I can go unconciouss, and there is a risk of falling into a coma.

## Analysis and Data exploration
In this section we look at the raw data a bit... We propose the $\Gamma$-hypothesis based on visuals and physical grounds.

## Models
In this section we introduce a few models which we will try to fit.

### Model 1:
Let $g(t)$ denote the blood glucose level at time $t$.  Assume insulin is administered at times $\{ t_{i,k} \}_{k=1}^{N_i}$ and food intake at times $\{ t_{f,k} \}_{k=1}^{N_f}$
the most simple model which adpots the $\Gamma$-hypothesis would be

$$
    g(t) = g(0) + c_i \left( \sum_{t_{i,k} < t } \int_{t_{i,k}}^t \frac{\tau}{\lambda_i} e^{-\tau / \lambda_i} d\tau \right)
        + c_f \left( \sum_{t_{f,j} < t } \int_{t_{f,j} }^t \frac{\tau}{\lambda_f} e^{-\tau / \lambda_f } d\tau \right) + Noise.
$$
This might be a good model to try first.  One critique is that it allows for the implausible cases where $g(t) < 0$.

### Model 2:
To prohibit negative glucose predictions we could introduce the auxiliary variable $x(t) = \log(g(t))$ and consider the model:
$$
    x(t) = x(0) + c_i \left( \sum_{t_{i,k} < t } \int_{t_{i,k}}^t \frac{\tau}{\lambda_i} e^{-\tau / \lambda_i} d\tau \right)
        + c_f \left( \sum_{t_{f,j} < t } \int_{t_{f,j} }^t \frac{\tau}{\lambda_f} e^{-\tau / \lambda_f } d\tau \right) + Noise.
$$
One could still make a similar critique.  Glocuse reading can only be so high: eventually physical process, such as the blood's chemical saturation level prevent higher glucose levels.

### Model 3:
To prohibit unusually high glucose readings we consider a function which diverges to infinity at $600$ mg/dL and diverges to negative infinity at zero.  For example, we could consider a rescaled and offset `arctanh` function.
We will consider the auxiliary variable
$$
    x(t) = {\rm arctanh}\left( \frac{2 g(t)}{g_{\rm max} } -1 \right)
$$
and again consider the model:
$$
    x(t) = x(0) + c_i \left( \sum_{t_{i,k} < t } \int_{t_{i,k}}^t \frac{\tau}{\lambda_i} e^{-\tau / \lambda_i} d\tau \right)
        + c_f \left( \sum_{t_{f,j} < t } \int_{t_{f,j} }^t \frac{\tau}{\lambda_f} e^{-\tau / \lambda_f } d\tau \right) + Noise.
$$

### Models 4,5,6:
I notice that my insuling sensitivity at Lunch is typically lower than my insulin sensitivity at breakfast.  Perhaps I should break the day into three parts, and consider difference coefficients for each time of day.


## Implementation
We desire to fit the parameter tuple $\theta = (\lambda_i,\lambda_f,c_i,c_f)$ to our data.
Our data is given by trajectories $x_\alpha(t)$ for $\alpha = 1,\dots,N$.
For each model we consider the cost function
$$
    C(\theta) = \sum_{\alpha=1}^N \| x_\alpha - x_{pred,\alpha}( \theta ) \|_{L_1}
$$
where the predicted trajectory is given by
$$
    x_{pred,\alpha}(\theta)(t) = x_\alpha(0) + c_i \left( \sum_{t_{i,k}<t} G(t,\lambda_i) \right) +  c_f \left( \sum_{t_{i,k}<t} G(t,\lambda_f) \right)
$$
We preform minimization using scipy's optimization module.

## Results
We did okay. Need more data probably

## Conclusion
Customized medicine